# Golf Flag Image Classification

## Purpose
The whole purpose of this notebook is for the creation, training and testing of an image classification model using Tensorflow Keras to classify if a golf flag is within an image.

Note: Important to acknowledge that this model was initially coded based on Nicholas Renotte video in youtube: [Image Classification Video](https://www.youtube.com/watch?v=jztwpsIzEGc&t=288s)

`-- The code has changed and I have adapted it to my needs.`

#### Steps
- Install Dependencies and set up
- Load Data
- Preprocess Data
- Create Deep Learning Model
- Evaluate Performance
- Save Model

## Installing Dependencies

In [ ]:
%pip install tensorflow tensorflow-gpu opencv-python matplotlib

In [ ]:
# importing dependencies
from matplotlib import pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os

In [ ]:
# Avoid OOM errors by setting GPU Memory Consuption Growth
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    # telling tensorflow do not use all the GPU memory, only use what you need
    tf.config.experimental.set_memory_growth(gpu, True)

#### Quick Note if you want to display an image through code


In [ ]:
# 1. Get the image file path
image_path = "" # some image file path

# 2. Read the image with cv2 (opencv)
img = cv2.imread(image_path)

# 3. show it with matplot lib (using pyplot)
plt.imshow(img) # if you want to color correct it do: plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

## Load Data

In [ ]:
# load the dataset from directory using keras
# if you want to see the documentation for it run: tf.keras.utils.image_dataset_from_directory?? 
data = tf.keras.utils.image_dataset_from_directory()

# it will by default:
# - Set the batch size to 32
# - Set the image_size to be (256, 256) 
# - Shuffle them

# the code above produces a generator, then we need to convert it to an iterator
data_iterator = data.as_numpy_iterator() # allow to access the elements to iterate

# if you want to get a batch of the images do: batch = data_iterator.next()
# a batch contains two elements, the image representation (image matrix as numpy arrays) and
# the second is the labels (the category of elements in numbers of 0 to more, ex 0 and 1 if there are only two categories)

# if we do not know how to flag what label is set to each category you can run the script below which
# will show the elements with the labels on top (helpful if you have more than one label)
# fig, ax = plt.subplots(ncols=4, figsize=(20,20))
# for idx, img in enumerate(batch[0][:4]):
#     ax[idx].imshow(img.astype(int)) # if the images are black then only use img and not the as type function
#     ax[idx].title.set_text(batch[1][idx])


## Preprocess Data
For image data we tend to preprocess it. We want to scale the image values to be between 0 and 1 (instead of 0 to 255). This helps the deep learning model generalize faster and produces better results. We are also going to split the data into training, testing and validation data partitions. This will allow us to ensure that we don't overfit the model.

#### Explaining **Overfitting**:
Overfitting occurs when a ML model is excessively complex and captures the noise and random fluctuations in the training data, rather than the underlying patterns that we want to learn from. If a model is overfitted it performs exceptionally well on the training data but poorly on new, unseen data.

Causes:
- Models with too many parameters can easily overfit
- Models with insufficient data to learn from, relies in noise and outliers --> outfitting the model
- Unclean data leads to overfitting since it will contain errors or random fluctuations that mislead the model

How to prevent overfitting:
- L1 and L2 Regularization penalize complex models (research on it later)
- Dividing data into training and validation sets (cross-validation, which is what we are doing in this model) to assess model performance
- Removing irrelevant or dedundant features
- Creating new training data by applying transformations to existing data (Data Augmentation)

In [ ]:
# Scaling data - as it goes from 0 to 255, to make it from 0 to 1, we need to divide by 255
data = data.map(lambda x,y: (x/255, y)) # in this case x being the image value and y the label

# to test with the batch (from before in comments) you can do:
# scaled = batch[0]/255
# scaled.min() # --> should be 0
# scaled.max() # --> should be 1

# to see that the mapping has happened inside the data pipeline (inside the data object)
# run data.as_numpy_iterator().next() and see the values between 0 and 1

In [ ]:
# Split Data - into train, test, and validation
# get the actual data parition sizes
train_size = int(len(data)*0.7) # ~70%
validation_size = int(len(data)*0.2) + 1 # ~20%
test_size = int(len(data)*0.1) + 1 # ~10%

# split the actual data
train_dataset = data.take(train_size)
validation_dataset = data.skip(train_size).take(validation_size)
test_dataset = data.skip(train_size + validation_size).take(test_size)

## Deep Learning Model
Build a deep learning model using the Keras Sequential API. 

Note: I would suggest reading about this API and understanding the type of model that is actually being built.

In [ ]:
# import dependencies for the model itself
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten

# Sequential (great for one data input and one data output) - flowing from top to bottom
# Functional (really powerful for multiple inputs, outputs, connections, more fancy stuff and etc.)

# layers:
# - Conv2D layer --> Convolutional Layer (Convolutional Neural Network...)
# - MaxPooling2D --> Acts as a condensing layer, goes through the images and actually 
#                    condenses them down (max or min value in a region)
# - Dense layer (fully connected layer) --> 
# - Flatten layer --> allows to go from a convolutional layer that contains multiple channels or kernels and reduces into 
#                     a format that our dense layer will be able to take
# - Dropout --> typically used for regularization

# when creating a deep learning model, the architecture of the model is composed of these layers

In [ ]:
# instantiate the Sequential model 
model = Sequential()

# chain the layers to the model to create the given architecture (where the magic happens)

# first section - convolution input layer
# 16 filters that scan over the image, each filter is 3px by 3px in size with a stride of 1
# stride of 1 means is moving one pixel at a time
# relu activation means that we are taking the output of this convolutional layer 
# relu function dictamines that each value that was below 0 now will be set to 0 (and preserve the positive values)
# input shape is set to be 256px by 256px by 3 channels (only set on first layer)

model.add(Conv2D(16, (3,3), 1, activation="relu", input_shape=(256,256,3))) 
model.add(MaxPooling2D()) # take the max values after the relu activation and return that value (default going 2 by 2)

# another conv later with 32 filters with size 3px by 3px
model.add(Conv2D(32, (3,3), 1, activation="relu"))
model.add(MaxPooling2D()) # scan again

model.add(Conv2D(16, (3,3), 1, activation="relu"))
model.add(MaxPooling2D())

model.add(Flatten()) # flattening the channels into a single one

# fully connected layers
model.add(Dense(256, activation="relu")) # 256 neurons performing a relu activation
model.add(Dense(1, activation="sigmoid")) # single neuron, single output after sigmoid activation have values from 0 to 1

In [ ]:
# then we need to compile this architecture
model.compile("adam", loss=tf.losses.BinaryCrossentropy(), metrics=["accuracy"])
# here "adam" is the optimizer (there are a ton of them)
# loss in this particular case will be BinaryCrossentropy since we are doing a binary classification problem
# tracking accuracy (how good is the model at classifying)

# if you want to look at the summary do: model.summary()
# the model summary provides the architecture of the model (and see how it transforms the data)
# the total number of params, trainable params and non-trainable params

In [ ]:
# Train the model
# set a directory for the logs of the model after training
logdir = "logs"

# create a callback, really useful to save model at specific checkpoints and do some logging (which is what we are doing)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

# fit our model (actually train it)
hist = model.fit(train_dataset, epochs=20, validation_data=validation_dataset, callbacks=[tensorboard_callback])

# explanation of above script:
# takes a training data, then it takes the number of epochs. Each epoch is one run over our entire train data set
# pass the validation data, it runs evaluation on the validation data to see how is the model performing in real time
# to save the train into a variable, in this case call hist (history), we can get all training information and validation
# data

# the loss in the training data (you want to see it go down)
# the accuracy in the training data (you want to see it go up)

# note: the training it will take the most time

In [ ]:
# plot performance (we can since we saved the variable hist)
# hist.history contains loss and accuracy information, let's plot it

# plot the loss
fig = plt.figure()
plt.plot(hist.history["los"], color="teal", label="loss")
plt.plot(hist.history["val_loss"], color="orange", label="val_loss")
fig.suptitle("Loss", fontsize=20)
plt.legend(loc="upper left")
plt.show()

# if loss is going down and validation loss going up, indication of overfitting
# might need to apply some regularization or change some data

# if the loss is nos steadily decreasing over time, it might mean that the model itself
# is not able to learn accordingly to predict the data. This leads to perhaps changing
# the architecture of the model itself

In [ ]:
# plot the accuracy
fig = plt.figure()
plt.plot(hist.history["accuracy"], color="teal", label="accuracy")
plt.plot(hist.history["val_accuracy"], color="orange", label="val_accuracy")
fig.suptitle("Accuracy", fontsize=20)
plt.legend(loc="upper left")
plt.show()

# it should be increasing over time (ideally using more and more data)

## Evaluating Performance
Time to test the model. We want to track how the model does and describe its precision, recall and accuracy. These are typical metrics used for classification problems.

#### Understanding performance metrics
- **Precision**:
    - 

- **Recall**:
    - 

- **Accuracy**:
    - 

In [ ]:
# imoprt more metric dependencies
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy 

In [ ]:
# instantiate the objects
precision_metric = Precision()
recall_metric = Recall()
accuracy_metric = BinaryAccuracy()

In [ ]:
# actually run the testing with the given performance metrics
for batch in test_dataset.as_numpy_iterator():
    x, y = batch # x being the image content and y the label
    
    # realize prediction with trained model
    yhat = model.predict(x)
    precision_metric.update_state(y, yhat)
    recall_metric.update_state(y, yhat)
    accuracy_metric.update_state(y, yhat)

# print the actual results for the performance review (results between 0 and 1)
print(f"Precision Metric: {precision_metric.result().numpy()} - Recall Metric: {recall_metric.result().numpy()} - Binary Accuracy Metric: {accuracy_metric.result().numpy()}")

# research how to make a confusion matrix!

In [ ]:
# let's test manully with real images!

# get the image file path on your machine
image_to_test = ""

# display image (to see what are we testing with)
img_test = cv2.imread(image_to_test)
plt.imshow(img_test) # remember to do the color correction if needed
plt.show()

# prepare the image to be tested
resized_image = tf.image.resize(img_test, (256, 256))
plt.imshow(resized_image) # remember to do color correction if needed
plt.show()

# predict (remember that it predicts upon a batch of images not a single one, then encapsulate it into a container (with np.expand_dims))
manual_predict_yhat = model.predict(np.expand_dims(resized_image/255, 0)) # at the same time we are scaling it dividing by 255

print(yhat) # will give the prediction output (remember to see what is 0 and what is 1)

## Save the model

In [ ]:
# save the model to use it again later
model.save(os.path.join("directory", "name_of_model.h5")) # remember to add the h5, also, put the directory that you want
# or just pass a file path that ends in h5

In [ ]:
# if you want to load the model and use it again
from tensorflow.keras.models import load_model
# new_model = load_model("filepath.h5") # here we just need the file path to that h5 model

# run a prediction with:
# new_model.predict(np.expand_dims(test_image/255, 0)) # this will do what we showed before